In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sc.version

'2.0.1'

In [11]:
from pyspark.sql import Row, SQLContext, HiveContext
from pyspark.sql import SparkSession
import time
import threading
import numpy as np
import random

In [4]:
input_path = 'hdfs://lattice-100.cs.colostate.edu:46780/test/southeast-0*-*.txt'
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

In [5]:
df = spark.read.json(input_path)

In [6]:
df.printSchema()
df.createOrReplaceTempView('noaa_data')

root
 |-- _corrupt_record: string (nullable = true)
 |-- location: string (nullable = true)
 |-- precipitable_water_entire_atmosphere: double (nullable = true)
 |-- relative_humidity_zerodegc_isotherm: double (nullable = true)
 |-- temperature_surface: double (nullable = true)
 |-- visibility_surface: double (nullable = true)



In [7]:
sql_stmt = 'SELECT * FROM noaa WHERE location LIKE "dn%" AND precipitable_water_entire_atmosphere > 8 AND precipitable_water_entire_atmosphere < 12'
sql_stmt2 = 'SELECT * FROM noaa_data WHERE temperature_surface > -38.80721 AND temperature_surface < -34.042206 AND relative_humidity_zerodegc_isotherm > 9.162598 AND relative_humidity_zerodegc_isotherm < 19.162598 AND precipitable_water_entire_atmosphere > 28.119974 AND precipitable_water_entire_atmosphere < 28.859655 AND location LIKE "dn%" AND visibility_surface > 7320.479 AND visibility_surface < 7580.649'
t1 = time.time()
spark.sql(sql_stmt2).count()
t2 = time.time()
print(t2-t1)

0
24.646201848983765


In [25]:
queries = [line.rstrip('\n') for line in open('sql-queries.txt')]
random.seed(123)
def rand_query():
    q = queries[random.randint(0, len(queries) - 1)]
    #print('Query: ', q)
    return q

In [30]:
def query(latencies):
  time.sleep(5)
  for j in range(5):
    t1 = time.time()
    print(spark.sql(rand_query()).count())
    t2 = time.time()
    latencies.append(t2 - t1)
    
#print(threading.currentThread().getName() ,': ',latencies)
#query([])

In [31]:
threads = []
latencies = []
thread_count = (1,1)
for t_c in thread_count:
    for i in range(t_c):
        t = threading.Thread(target=query, args=(latencies,))
        threads.append(t)
        t.start()
        if i == t_c - 1:
            t.join()
    lat_data = np.array(latencies)
    with open('/tmp/spark-sql-' + str(t_c) + '.csv', 'w') as file_handler:
        for item in latencies:
            file_handler.write("{}\n".format(item))
    print('Thread-Count: ', t_c,' Mean: ', np.mean(lat_data), ', Std. Dev: ', np.std(lat_data), '95th Percentile: ',np.percentile(lat_data, 95))

0
0
0
0
0
Thread-Count:  1  Mean:  20.5772511959 , Std. Dev:  1.76199753623 95th Percentile:  22.3994861126
0
0
0
0
0
Thread-Count:  1  Mean:  20.0877991438 , Std. Dev:  2.37266512613 95th Percentile:  23.4629499674
